### Notebook to create Initial dataset sample for the project with employees data

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, sum as spark_sum, when

In [ ]:
spark = SparkSession.builder \
    .appName("HR Dataset Analysis") \
    .getOrCreate()

In [ ]:
df = spark.read.format("csv").option("header",True).load("/content/drive/MyDrive/Estudos/Douglas College - Pós/Applied Research/HR_Data_MNC_Data Science Lovers.csv")

In [ ]:
df.printSchema()

root
 |-- Unnamed: 0: string (nullable = true)
 |-- Employee_ID: string (nullable = true)
 |-- Full_Name: string (nullable = true)
 |-- Department: string (nullable = true)
 |-- Job_Title: string (nullable = true)
 |-- Hire_Date: timestamp (nullable = true)
 |-- Location: string (nullable = true)
 |-- Performance_Rating: double (nullable = true)
 |-- Experience_Years: double (nullable = true)
 |-- Status: string (nullable = true)
 |-- Work_Mode: string (nullable = true)
 |-- Salary_INR: double (nullable = true)



In [ ]:
df = df.withColumn("Experience_Years", col("Experience_Years").cast("double")) \
        .withColumn("Salary_INR", col("Salary_INR").cast("double")) \
        .withColumn("Performance_Rating", col("Performance_Rating").cast("double")) \
        .withColumn("Hire_Date", col("Hire_Date").cast("timestamp"))


In [ ]:
df.show(5)

+----------+-----------+---------------+----------+--------------------+-------------------+--------------------+------------------+----------------+--------+---------+----------+
|Unnamed: 0|Employee_ID|      Full_Name|Department|           Job_Title|          Hire_Date|            Location|Performance_Rating|Experience_Years|  Status|Work_Mode|Salary_INR|
+----------+-----------+---------------+----------+--------------------+-------------------+--------------------+------------------+----------------+--------+---------+----------+
|         0| EMP0000001|  Joshua Nguyen|        IT|   Software Engineer|2011-08-10 00:00:00|  Isaacland, Denmark|               5.0|            14.0|Resigned|  On-site| 1585363.0|
|         1| EMP0000002| Julie Williams| Marketing|      SEO Specialist|2018-03-02 00:00:00|Anthonyside, Cost...|               2.0|             7.0|  Active|  On-site|  847686.0|
|         2| EMP0000003|Alyssa Martinez|        HR|          HR Manager|2023-03-20 00:00:00|Port Chr

In [ ]:
df.describe().show()

+-------+----------------+-----------+-------------+----------+--------------------+-------------------+------------------+------------------+----------+---------+------------------+
|summary|      Unnamed: 0|Employee_ID|    Full_Name|Department|           Job_Title|           Location|Performance_Rating|  Experience_Years|    Status|Work_Mode|        Salary_INR|
+-------+----------------+-----------+-------------+----------+--------------------+-------------------+------------------+------------------+----------+---------+------------------+
|  count|         2000000|    2000000|      2000000|   2000000|             2000000|            2000000|           2000000|           2000000|   2000000|  2000000|           2000000|
|   mean|        999999.5|       NULL|         NULL|      NULL|                NULL|               NULL|         3.0001485|          5.010287|      NULL|     NULL|    896887.7556635|
| stddev|577350.413527175|       NULL|         NULL|      NULL|                NULL| 

In [ ]:
sampled_df = df.sample(withReplacement=False, fraction=1.0, seed=4495).limit(5000)

In [ ]:
sampled_df.describe().show()

+-------+------------------+-----------+---------------+----------+--------------------+-----------------+------------------+------------------+----------+---------+------------------+
|summary|        Unnamed: 0|Employee_ID|      Full_Name|Department|           Job_Title|         Location|Performance_Rating|  Experience_Years|    Status|Work_Mode|        Salary_INR|
+-------+------------------+-----------+---------------+----------+--------------------+-----------------+------------------+------------------+----------+---------+------------------+
|  count|              5000|       5000|           5000|      5000|                5000|             5000|              5000|              5000|      5000|     5000|              5000|
|   mean|            2499.5|       NULL|           NULL|      NULL|                NULL|             NULL|            2.9614|            5.0364|      NULL|     NULL|       896535.1738|
| stddev|1443.5200033252052|       NULL|           NULL|      NULL|        

In [ ]:
sampled_df.groupBy("Job_Title").count().show(10000)

+--------------------+-----+
|           Job_Title|count|
+--------------------+-----+
|   Software Engineer|  762|
|      SEO Specialist|  182|
|          HR Manager|   62|
|Logistics Coordin...|  237|
|     Finance Manager|   87|
|     Account Manager|  269|
|   Financial Analyst|  153|
|     Sales Executive|  507|
|Operations Executive|  389|
|        HR Executive|  202|
|      Sales Director|   52|
|     DevOps Engineer|  234|
|  Content Strategist|   95|
| Marketing Executive|  314|
|  Research Scientist|  114|
|          Accountant|  211|
|          IT Manager|  136|
|Supply Chain Manager|  124|
|        Data Analyst|  275|
|Talent Acquisitio...|  131|
|   Product Developer|   75|
| Operations Director|   42|
|                 CTO|   76|
|Business Developm...|  138|
|      Lab Technician|   44|
|                 CFO|   31|
|       Brand Manager|   25|
|         HR Director|   21|
|  Innovation Manager|   12|
+--------------------+-----+



In [ ]:
percentiles = df.approxQuantile("Experience_Years", [0.25, 0.5, 0.75], 0.01)

p25 = 3
p50 = 8
p75 = 11

executive_titles = ["CTO", "CFO", "HR Director", "Operations Director", "Sales Director"]

sampled_df = sampled_df.withColumn(
    "Job_Level",
    when(col("Job_Title").isin(executive_titles), "Executive")  # top layer first
    .when(col("Experience_Years") <= p25, "Specialist")
    .when((col("Experience_Years") > p25) & (col("Experience_Years") <= p50), "Analyst")
    .when((col("Experience_Years") > p50) & (col("Experience_Years") <= p75), "Manager")
    .otherwise("Principal")  # remaining high-experience staff
)

print(f"25th: {p25}, 50th: {p50}, 75th: {p75}")

25th: 3, 50th: 8, 75th: 11


In [ ]:
sampled_df.select([spark_sum(col(c).isNull().cast("int")).alias(c) for c in df.columns]).show()

+----------+-----------+---------+----------+---------+---------+--------+------------------+----------------+------+---------+----------+
|Unnamed: 0|Employee_ID|Full_Name|Department|Job_Title|Hire_Date|Location|Performance_Rating|Experience_Years|Status|Work_Mode|Salary_INR|
+----------+-----------+---------+----------+---------+---------+--------+------------------+----------------+------+---------+----------+
|         0|          0|        0|         0|        0|        0|       0|                 0|               0|     0|        0|         0|
+----------+-----------+---------+----------+---------+---------+--------+------------------+----------------+------+---------+----------+



In [ ]:
sampled_df.groupBy("Job_Level", "Status").count().orderBy("Job_Level").show(10000)

+----------+----------+-----+
| Job_Level|    Status|count|
+----------+----------+-----+
|   Analyst|    Active| 1387|
|   Analyst|  Resigned|  369|
|   Analyst|Terminated|   96|
|   Analyst|   Retired|  114|
| Executive|    Active|  149|
| Executive|   Retired|   12|
| Executive|  Resigned|   52|
| Executive|Terminated|    9|
|   Manager|    Active|  430|
|   Manager|  Resigned|  113|
|   Manager|   Retired|   32|
|   Manager|Terminated|   28|
| Principal|  Resigned|   59|
| Principal|    Active|  216|
| Principal|   Retired|   10|
| Principal|Terminated|   12|
|Specialist|    Active| 1326|
|Specialist|  Resigned|  389|
|Specialist|   Retired|  102|
|Specialist|Terminated|   95|
+----------+----------+-----+



In [ ]:
sampled_df.write.option("header", True).mode("overwrite").csv("/content/drive/MyDrive/Estudos/Douglas College - Pós/Applied Research/initial_dataset.csv")

In [ ]:
sampled_df.groupBy("Status").count().show(10000)

+----------+-----+
|    Status|count|
+----------+-----+
|  Resigned|  982|
|    Active| 3508|
|Terminated|  240|
|   Retired|  270|
+----------+-----+

